In [1]:
import os
import platform
import logging

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# import matplotlib.pyplot as plt
import tqdm
import cv2
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.fr_detector import FRDetector

In [2]:
LOGGER = logging.getLogger(__name__)

In [3]:
if platform.system() == "Linux":
    model_path = "./street_signs_identify/detector/saved_models/frcnn.pth"
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")
    
else:
#     folder_name = "val"
#     model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
#     data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
#     output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf\\{folder_name}'
#     output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_gt\\{folder_name}'
#     output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_filter\\{folder_name}'
#     output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_craft\\{folder_name}'
    
    # demo
    model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
    data_path = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo3'
    output_folder = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo3\xf'
    output_folder_gt = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo3\xf_gt'
    output_folder_filtered_pred = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo3\xf_filtered_pred'
    output_folder_craft_pred = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo3\xf_f'
    



os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True) 
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [4]:
detector = FRDetector(model_ckpt=model_path)
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

In [5]:
x_image_loader._info.head()

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,220707_023525413_Camera_3.jpg,3Taoyuan_nan,1,"[(101, 932), (87, 1002), (269, 1042), (280, 986)]",木墊貨專區,"POLYGON ((101 932, 87 1002, 269 1042, 280 986,...",87,932,280,1042


In [6]:
global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "detected_index", "IOU"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]
    detected_image = detector(gt_image.image)
    if detected_image is None:
        LOGGER.warning("Detection failure: %s", image_name)
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        LOGGER.warning("Detected instance and GT are all disjoint: %s", image_name)
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    filtered_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = filtered_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))

    for row_idx, row in overlapping_info.iterrows():
        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "IOU"] = row["IOU"]
        label_df.loc[global_index, "gt_index"] = int(gt_index)
        label_df.loc[global_index, "detected_index"] = int(detected_index)

        crapped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1

for src_file, dg in label_df.groupby("src_file"):
    for detected_idx, sub_dg in dg.groupby("detected_index"):
        sorted_index = sub_dg["IOU"].sort_index().index
        # print(sorted_index)
        label_df.loc[sorted_index, "priority"] = range(len(sorted_index))

# for src_file, dg in label_df.groupby("src_file"):
#     print(dg)

label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.47s/it]
